In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import random
import matplotlib.pyplot as plt
import cv2
import time 
import keras.backend as K
import keras 
import json 
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.utils import plot_model
from keras.optimizers import Adam, SGD
from keras.losses import mean_absolute_error, categorical_crossentropy,mean_absolute_error
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Lambda, concatenate
from keras.models import Input, Model, load_model

Using TensorFlow backend.


## Global Parameters

In [2]:
TRAINING_SIZE = 100000
TESTING_SIZE = 2000
VALIDATION_SIZE = 2000

TOTAL_IMAGES = 180000
IMAGE_SIZE = 224

BATCH_SIZE = 8
TRIPLET_INDEX = 0

ENCODINGS_DIM = 1000

## Loading Dataset

In [3]:
arod = h5py.File('./AROD_HDF/AROD.hdf','r')
triplets = pd.read_csv('./triplets.csv').get_values()[0:TRAINING_SIZE]
training_set = triplets[:,1:4]
def get_triplet():
    global TRIPLET_INDEX
    triplet = training_set[TRIPLET_INDEX]
    
    a = arod['IMAGES'][triplet[0]]
    p = arod['IMAGES'][triplet[1]]
    n = arod['IMAGES'][triplet[2]]
    
    sa = arod['SCORES'][triplet[0]][0]        
    sp = arod['SCORES'][triplet[1]][0]        
    sn = arod['SCORES'][triplet[2]][0]        
    TRIPLET_INDEX = TRIPLET_INDEX + 1
    if TRIPLET_INDEX > 80000:
        TRIPLET_INDEX = 0 
    return a, p, n, sa, sp, sn 

def Generate():
    while True:
        list_a = []
        list_p = []
        list_n = []
        label = []

        for i in range(BATCH_SIZE):
            a, p, n, sa, sp, sn = get_triplet()
            list_a.append(a)
            list_p.append(p)
            list_n.append(n)
            label.append([sa,sn])
            
        A = preprocess_input(np.array(list_a, dtype = 'float32'))
        B = preprocess_input(np.array(list_p, dtype = 'float32'))
        C = preprocess_input(np.array(list_n, dtype = 'float32'))
        label = np.array(label,dtype = 'float32')
        return [A, B, C], label



## Loss function


In [4]:
def identity_loss(y_true, y_pred):
    r = y_true[0] - y_pred[0]
    #return K.mean(y_pred - 0 * y_true)
    return K.sum(y_pred - 0 * y_true,axis=-1)

def Le(X):
    a, p, n = X
    m = 0.2 * 100
    loss = K.relu(m + K.sum(K.square(a-p),axis=-1,keepdims=True) - K.sum(K.square(a-n),axis=-1,keepdims=True))
    return loss

def Ld_1(X):
    a, p, n = X
    m = 0.3 * 100
    loss = K.relu(m+ K.sqrt(K.sum(K.square(a),axis=-1,keepdims=True)) - K.sqrt(K.sum(K.square(n),axis=-1,keepdims=True)))
    return loss

def triplet_loss(y_true,y_pred):
    sa = y_true[0]
    sp = y_true[1]
    sn = y_true[2]
    ld = y_pred[0]
    le = y_pred[1]
    return (sn - sa)*ld + le


def fake_triplet_loss(y_true,y_pred):
    sa = y_true[:,0]
    sn = y_true[:,1]
    #sn = y_true[2]
    
    le = y_pred[:,0]
    ld = y_pred[:,1]
    
    return (sn - sa)*ld + le

## Loading Model

In [34]:
def GetBaseModel():
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    dense_1 = Dense(ENCODINGS_DIM,activation='sigmoid')(x)
    base_model = Model(base_model.input,dense_1, name="base_model")
    return base_model

def GetModel(base_model):
    input_1 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_2 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_3 = Input((IMAGE_SIZE,IMAGE_SIZE,3))

    r1 = base_model(input_1)
    r2 = base_model(input_2)
    r3= base_model(input_3)

    loss_le = Lambda(Le)([r1,r2,r3])
    loss_ld1 = Lambda(Ld_1)([r1,r2,r3])
    loss = concatenate([loss_le,loss_ld1],axis=-1)
    
    
    trainedModel = Model(inputs=[input_1, input_2, input_3], outputs=loss)
    trainedModel.compile(loss=identity_loss, optimizer=Adam(0.000003))
    
    
    
    trainedModel.load_weights(filepath='./model.h5')
    
    
    
    basemodel_trained_layers = trainedModel.layers[3].layers
    testmodel = GetBaseModel()
    
    for i in range(len(basemodel_trained_layers)):
        try:
            weights = basemodel_trained_layers[i].get_weights()
            testmodel.layers[i].set_weights(weights)
            print i
        except:
            print "layer "+str(i) +" with problems"
    return testmodel


def GetModelM(base_model):
    input_1 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_2 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_3 = Input((IMAGE_SIZE,IMAGE_SIZE,3))

    r1 = base_model(input_1)
    r2 = base_model(input_2)
    r3= base_model(input_3)
    return Model(inputs=[input_1, input_2, input_3], outputs=[r1,r2,r3])


model = GetModel(GetBaseModel())
model.summary()
#model = GetModelM(GetBaseModel())
#model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [6]:
62500/12

5208

In [35]:
[A,B,C],labels = Generate()
#scoresvector = model.predict(preprocess_input(np.array(arod['IMAGES'][:10],dtype='float32')),batch_size=32,verbose=1)
scoresvector = model.predict(A,batch_size=32,verbose=1)

8/8 [==============================] - 5s 573ms/step


In [36]:
scoresvector


array([[0.6207594 , 0.6932216 , 0.45252618, ..., 0.6716497 , 0.29566443,
        0.39145863],
       [0.4299156 , 0.20265882, 0.41470283, ..., 0.6688448 , 0.2521137 ,
        0.8448964 ],
       [0.12980983, 0.43074664, 0.6566168 , ..., 0.8962864 , 0.50282377,
        0.81529653],
       ...,
       [0.38022688, 0.7000111 , 0.33483687, ..., 0.8523079 , 0.62095016,
        0.41873965],
       [0.5351693 , 0.67404556, 0.39761493, ..., 0.7846613 , 0.3608017 ,
        0.796291  ],
       [0.56303924, 0.17892121, 0.7323153 , ..., 0.688025  , 0.3322742 ,
        0.68848914]], dtype=float32)

In [15]:
sess = tf.Session()
Le_out = sess.run (Le(scoresvector))
Ld_out = sess.run (Ld_1(scoresvector))


In [18]:
Le_out.c

array([[466.27844],
       [548.3136 ],
       [226.93726],
       [170.0459 ],
       [773.86456],
       [  0.     ],
       [694.9888 ],
       [ 89.14661]], dtype=float32)

In [16]:
Ld_out

array([[18.26286 ],
       [36.927013],
       [23.853527],
       [40.172085],
       [28.780113],
       [24.47107 ],
       [46.731354],
       [29.398834]], dtype=float32)

In [17]:
labels

array([[0.4047688 , 0.32029647],
       [0.57713735, 0.1084412 ],
       [0.17069401, 0.56759614],
       [0.36863664, 0.5318241 ],
       [0.32091293, 0.5836469 ],
       [0.45853236, 0.3518203 ],
       [0.31629398, 0.4399937 ],
       [0.5551203 , 0.126861  ]], dtype=float32)

In [26]:
LLL =np.append(Le_out,Ld_out,axis=-1)

In [28]:
LLL

array([[466.27844 ,  18.26286 ],
       [548.3136  ,  36.927013],
       [226.93726 ,  23.853527],
       [170.0459  ,  40.172085],
       [773.86456 ,  28.780113],
       [  0.      ,  24.47107 ],
       [694.9888  ,  46.731354],
       [ 89.14661 ,  29.398834]], dtype=float32)

In [29]:
fake_triplet_loss_out = fake_triplet_loss(y_true=labels,y_pred=LLL)

In [30]:
fake_triplet_loss_out


array([464.73575  , 531.00604  , 236.40477  , 176.60149  , 781.4261   ,
        -2.6113586, 700.7694   ,  76.55628  ], dtype=float32)

In [ ]:
scores = np.sum(scoresvector,axis=-1)

In [ ]:
scores

In [ ]:
model.save_weights('./trained_weights.h5py')

In [ ]:
arod['IMAGES'][97192]

In [ ]:
preprocess_input(np.array(arod['IMAGES'][97192],dtype = 'float32'))

In [ ]:
model.summary()

In [69]:
def codification_vector_norm (vector):
    return np.sqrt(np.sum(np.square(vector),axis=-1))/(1000**.5)

In [85]:
codification_vector_norm(scoresvector)

array([[0.5488429 , 0.5580086 , 0.53882444, 0.53292614, 0.55104905,
        0.5455094 , 0.5617372 , 0.54152834],
       [0.5500357 , 0.53417504, 0.5392807 , 0.53851026, 0.53978086,
        0.5526381 , 0.5585005 , 0.54588455],
       [0.5379609 , 0.5435106 , 0.52263916, 0.543875  , 0.53962034,
        0.5404179 , 0.52969885, 0.55705744]], dtype=float32)

In [86]:
np.shape(scoresvector)

(3, 8, 1000)